# Svenskt Kvinnobiografiskt lexikon 

* Wikidataproperty [P4963](https://www.wikidata.org/wiki/Property_talk:P4963)

In [70]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
 
url = "https://ws.spraakbanken.gu.se/ws/karp/v5/minientry?q=extended%7C%7Cand%7Cnamn%7Cexists&mode=skbllinks&show=name%2Curl&sort=sorteringsnamn.sort%2Csorteringsnamn.init%2Ctilltalsnamn.sort&size=10000&resource=skbl"
r = http.request('GET', url) 
data = json.loads(r.data.decode('utf-8')) 
#print(type(data))
#print(r.info())
for row in (data["hits"]["hits"]):
#    print (row["_id"])
#    print (row["_index"])
    print (row["_source"])

#    print (row)
#df2 = pd.read_json()
df = pd.DataFrame(data["hits"]["hits"]["_source"])

{'name': {'firstname': 'Margareta (/Margit/) Ulrika', 'lastname': 'Abenius', 'sortname': 'Abenius'}, 'url': 'MargitAbenius'}
{'name': {'firstname': '/Euphrosyne/', 'lastname': 'Abrahamson', 'sortname': 'Abrahamson'}, 'url': 'EuphrosyneAbrahamson'}
{'name': {'firstname': '/Alma/ Sofia', 'lastname': 'Abrahamsson', 'sortname': 'Abrahamsson'}, 'url': 'AlmaAbrahamsson'}
{'name': {'firstname': 'Ingela /Margareta/', 'lastname': 'Abrahamsson', 'sortname': 'Abrahamsson'}, 'url': 'IngelaMargaretaAbrahamsson'}
{'name': {'firstname': '/Wilhelmina/ Sofia', 'lastname': 'Acrel', 'sortname': 'Acrel'}, 'url': 'WilhelminaSofiaAcrel'}
{'name': {'firstname': '/Karin/ Charlotta Teresia', 'lastname': 'Adamsson', 'sortname': 'Adamsson'}, 'url': 'KarinAdamsson'}
{'name': {'firstname': 'Eleonora Amalia /Maria/', 'lastname': 'Adelborg', 'sortname': 'Adelborg'}, 'url': 'MariaAdelborg'}
{'name': {'firstname': 'Eva /Ottilia/', 'lastname': 'Adelborg', 'sortname': 'Adelborg'}, 'url': 'OttiliaAdelborg'}
{'name': {'fi

TypeError: list indices must be integers or slices, not str

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1491 entries, 0 to 1490
Data columns (total 6 columns):
_id        1491 non-null object
_index     1491 non-null object
_score     0 non-null object
_source    1491 non-null object
_type      1491 non-null object
sort       1491 non-null object
dtypes: object(6)
memory usage: 70.0+ KB


In [58]:
df

,_id,_index,_score,_source,_type,sort
0,AVhiQo6bUc4E-N8JPwxO,skbl_20200508-1305,None,{'name': {'firstname': 'Margareta (/Margit/) U...,entry,"[ᒕ䱐未ᓐࡘЬ  , a, ₔ勍┩㌔祊ҥᙪ㥏⢟勎➥ထ䠈搀]"
1,AVSa7OunyLWGQ0YXKrQI,skbl_20200508-1305,None,"{'name': {'firstname': '/Euphrosyne/', 'lastna...",entry,"[ᒕ勊⛥ᐔ横ఄᰂ฀, a, ᢨ凍楨哕䨙䐄ᰂ฀]"
2,AV-6FkE-WxymAE2pjnq5,skbl_20200508-1305,None,"{'name': {'firstname': '/Alma/ Sofia', 'lastna...",entry,"[ᒕ勊⛥ᐔ橪ᔌȞď , a, ᒟ偊⦨匳䥈дȚ  ]"
3,gJmcw3ABqTT99FE_vQpY,skbl_20200508-1305,None,"{'name': {'firstname': 'Ingela /Margareta/', '...",entry,"[ᒕ勊⛥ᐔ橪ᔌȞď , a, Ს䵌⟥ᐒ䩙咥ᙢ伩䁄怀]"
4,95n_EXIBqTT99FE_TxCd,skbl_20200508-1305,None,"{'name': {'firstname': '/Wilhelmina/ Sofia', '...",entry,"[ᒖ勌⟠႐ࡈ  , a, ⪜俍昧琓䨙┵੦㤩䁄怀]"
...,...,...,...,...,...,...
1486,Q3DZ6XIBTDNbi1A-7ESn,skbllinks_20200625,None,{'name': {'firstname': '/Martina/ Sofia Helena...,entry,"[⸷䄓⧦ᒲ妊ⳕɒ䭏Ს䩀⊀ᅀ  , ö, ₔ勓木㊔権䳤剮ㄿᢡ䩀⋀ᅠ  ]"
1487,AV9X8oMlWxymAE2pjnn4,skbl_20200508-1305,None,{'name': {'firstname': 'Anna Lovisa (/Anna-Lis...,entry,"[⸷䄓⧧犔㨹糥ᩒď䀀, ö, ᒡ僊⟨唳䩩⒥چ⤿Ღ䩀⋀ᅠ  ]"
1488,AWEDUXhGWxymAE2pjnyp,skbl_20200508-1305,None,"{'name': {'firstname': '/Barbro/ Margareta', '...",entry,"[⸷䄓⧧玓㨘ЬȖ  , ö, ᖔ勊楨吒䩙咥ᙢ伩䁄怀]"
1489,AVsaKKo2i-B3PHNCX9Sm,skbl_20200508-1305,None,"{'name': {'firstname': '/Agda/ Maria', 'lastna...",entry,"[⸷䄓⧧甔᥸ЬȖ  , ö, ᒚ䯊⠥ᒳ䥈дȚ  ]"
